# Estimate Binomial draw probabilility

In [1]:
using SRDynamicHMC

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-2.6548056865833978

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0012 s/step ...done
MCMC, adapting ϵ (25 steps)
6.5e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0024 s/step ...done
MCMC, adapting ϵ (100 steps)
7.6e-6 s/step ...done
MCMC, adapting ϵ (200 steps)
7.6e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
9.5e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
1.1e-5 s/step ...done
MCMC (1000 steps)
6.6e-6 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.552877], -5.713, 2, DoubledTurn, 0.648339, 3), NUTS_Transition{Array{Float64,1},Float64}([0.659323], -3.06611, 2, DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.836795], -3.16046, 2, DoubledTurn, 0.967635, 3), NUTS_Transition{Array{Float64,1},Float64}([1.91229], -6.4469, 2, DoubledTurn, 0.468474, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.0256905], -5.82093, 1, AdjacentTurn, 0.820905, 3), NUTS_Transition{Array{Float64,1},Float64}([0.670406], -3.74544, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.294183], -3.42611, 1, DoubledTurn, 0.866748, 1), NUTS_Transition{Array{Float64,1},Float64}([0.327323], -3.55702, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.767162], -3.23097, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.700398], -2.96015, 1, DoubledTurn, 0.988773, 1)  …  NUTS_Transition{Array{Float64,1},

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

362.85859910827355

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.19 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 34% DoubledTurn => 66%
  depth: 1 => 64% 2 => 36% 3 => 0%


check the mean

In [11]:
mean(posterior_α)

0.7316171626811596

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*